# Tic-Tac-Toe by Descending the Game Play Tree

A simple game of tic-tac-toe can be encoded in a 3x3 matrix like below. With `-1` and `+1` as X and O respectively.

> Actually used it the other way around but that shouldn't be much of a problem


In [2]:
import numpy as np

board = np.array([[-1, +1,  0],
                  [0,  -1, +1],
                  [0,  -1, +1]])

**Problem**: Given a matrix like above, write a function that determines if there are empty tiles on the board. Look at [`np.any`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.any.html)

In [3]:
def open(board):
    return np.any(board==0)

print(open(board))

True


**Problem:** Write a similar function to see who has won the game. Return -1, 1, 0 for -1, +1, or nobody winning respectively.

In [8]:
def check(board):
    boardSumsDown = board.sum(0)
    # Check for down winner
    if(np.any(boardSumsDown == -3)):
        return -1
    if(np.any(boardSumsDown == 3)):
        return 1
    boardSumsAcross = board.sum(1)
    # Check for across winner
    if(np.any(boardSumsAcross == -3)):
        return -1
    if(np.any(boardSumsDown==3)):
        return 1
    # There's still no winner, check the diagonals
    diagonala = board[0,0] + board[1,1] + board[2,2]
    diagonalb = board[0,2] + board[1,1] + board[2,0]
    if(diagonala == -3 or diagonalb == -3):
        return -1
    if(diagonala == 3 or diagonalb == 3):
        return 1
    return 0
    
check(board)

0

Given a board with open spaces, we can add +1 or -1 to any of the open spaces to generate a possible next turn state.

**Problem:** Given that it is `player`'s turn, generate a list of all the possible next game states for all the possible moves `player` could take.

In [6]:
def move(board, player):
    # Create new array with all possibilities
    boards = []
    for i in range(board.shape[0]):
        # Iterating through column
        for j in range(board.shape[1]):
            # Iterating through rows
            if(board[j,i] == 0):
                # If current spot = 0, create a new board and add the number
                newBoard = board.copy()
                newBoard[j,i] = player
                boards.append(newBoard)
    return boards

If a player wishes to take any given move, we ought to rank the possible moves somehow.

**Problem:** Think of a possible way to give a score for a given board that correlates to: high score ==> board will likely win, low score ==> board will likely lose. Implement this score. Be sure to take into consideration future game states.

**Solution:**

*oh boy is it a bad one*

**If there's a winner, it returns 10. If they can't win, it returns -10.**

Step One: Wins down a path are worth 0.75/steps down, losses are worth -0.6/steps down, this makes sure that closer things are prioritized. Paths are measured by going down the tree and finding numbers

Step Two: The wins and losses are added

Example:
Loss two down: -0.3, 
Two wins three down: 0.5, 
Loss, win four down: 0.0375

Score for path: 0.2375

Another path could have an early win, but then many losses after.

Win one down: 0.75
Loss two down: -0.3
Two losses three down: -0.2
Win, loss four down: 0.0375

Score for path: 0.2875

This path is slightly better because of an early win.

In [58]:
def score(board, player):
    # go through
    s = 0.0
    moves = move(board, player)
    # checks for winner
    for i in range(len(moves)):
        if(check(moves[i]) == player):
            # winner move found
            s = s + 4
        else:
            # winner not found
            moves2 = move(board, player*-1)
            for j in range(len(moves2)):
                # all of the moves that opponent can make
                if(check(moves2[j]) == player*-1):
                    # winners found for opponent
                    return -10
                else:
                    # winner not found on this path
                    if(check(moves[i]) == player):
                        s = s + 0.5
                    else:
                        # winner not found
                        moves2 = move(board, player*-1)
                        for j in range(len(moves2)):
                            # all of the moves that opponent can make
                            if(check(moves2[j]) == player*-1):
                                # winners found for opponent
                                s = s - 0.87
    return s

score(board, 1)
print("")
print("")
print("")
score(board, -1)
                    

-10

**Problem:** Implement a function so you can play the game. Display the board each turn and give the board a score. Use `input()` to collect user input for game play. Make sure your `score` makes intuitive sense. [Result](http://www.dictionary.com/browse/tragedy?s=t)

In [60]:
def human_user(board, player):
    """
    For a board and a player, get the next game state by using `input` to get the players move
    
    return the board for the next game state
    """
    user = input("Going down by columns, pick the number zero you'd like to occupy (1-9)")
    board = move(board, player)[int(user)-1]
    return board
    
def play(player1_input=human_user, player2_input=human_user):
    """
    User player1_input and player2_input to get the next game state.
    
    play until someone has won the game.
    """
    rge = np.array([[0, 0,  0],
            [0,  0, 0],
             [0,  0, 0]])
    winner = False
    currentlyUp = 1
    while(winner == False):
        if(currentlyUp == 1):
            rge = player1_input(rge,1)
        else:
            rge = player2_input(rge,-1)
        print(rge)
        if(check(rge) == 1):
            winner = True
            print("Winner is player 1")
        if(check(rge) == -1):
            winner = True
            print("Winner is player 2")
        if(open(rge)==False):
            winner = True
            print("tie")
        print("")
        currentlyUp = currentlyUp*-1
play()

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[1 0 0]
 [0 0 0]
 [0 0 0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1  0  0]
 [-1  0  0]
 [ 0  0  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1  0  0]
 [-1  0  0]
 [ 1  0  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1 -1  0]
 [-1  0  0]
 [ 1  0  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1 -1  0]
 [-1  1  0]
 [ 1  0  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1 -1  0]
 [-1  1  0]
 [ 1 -1  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1 -1  1]
 [-1  1  0]
 [ 1 -1  0]]
Winner is player 1



**Problem:** Implement an AI user that given the board and player, picks the next game state board with the highest score.

In [56]:
def ai_user(board, player):
    testing = move(board, player)
    indexWithHighestScore = 0
    for i in range(len(testing)):
        if(score(testing[i],player) > score(testing[indexWithHighestScore],player)):
            indexWithHighestScore = i
    print(score(testing[indexWithHighestScore], player))
    return testing[indexWithHighestScore]

**Problem:** Play the game against the AI.

In [67]:
play(player1_input=ai_user)

0.0
[[1 0 0]
 [0 0 0]
 [0 0 0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)4
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  0]]

4.0
[[ 1  0  0]
 [ 1 -1  0]
 [ 0  0  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1  0  0]
 [ 1 -1  0]
 [-1  0  0]]

0.0
[[ 1  0  1]
 [ 1 -1  0]
 [-1  0  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1 -1  1]
 [ 1 -1  0]
 [-1  0  0]]

0.0
[[ 1 -1  1]
 [ 1 -1  0]
 [-1  1  0]]

Going down by columns, pick the number zero you'd like to occupy (1-9)1
[[ 1 -1  1]
 [ 1 -1 -1]
 [-1  1  0]]

0.0
[[ 1 -1  1]
 [ 1 -1 -1]
 [-1  1  1]]
tie

